# Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# EDA

In [3]:
df = pd.read_csv('train.csv', index_col='id')
df.head()

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,breathing_restriction,toe_inflammation,finger_inflammation,lips_irritation,itchiness,ulcers,toenail_loss,speech_problem,bullseye_rash,prognosis
id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lyme_disease
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Tungiasis
2,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,Lyme_disease
3,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Zika
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,Rift_Valley_fever


In [4]:
df.isna().sum()

sudden_fever      0
headache          0
mouth_bleed       0
nose_bleed        0
muscle_pain       0
                 ..
ulcers            0
toenail_loss      0
speech_problem    0
bullseye_rash     0
prognosis         0
Length: 65, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 707 entries, 0 to 706
Data columns (total 65 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sudden_fever           707 non-null    float64
 1   headache               707 non-null    float64
 2   mouth_bleed            707 non-null    float64
 3   nose_bleed             707 non-null    float64
 4   muscle_pain            707 non-null    float64
 5   joint_pain             707 non-null    float64
 6   vomiting               707 non-null    float64
 7   rash                   707 non-null    float64
 8   diarrhea               707 non-null    float64
 9   hypotension            707 non-null    float64
 10  pleural_effusion       707 non-null    float64
 11  ascites                707 non-null    float64
 12  gastro_bleeding        707 non-null    float64
 13  swelling               707 non-null    float64
 14  nausea                 707 non-null    float64
 15  chills     

In [6]:
df.shape

(707, 65)

### Train test split

In [7]:
from sklearn.model_selection import train_test_split
X = df.drop('prognosis',axis=1)
y = df['prognosis']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((565, 64), (142, 64), (565,), (142,))

# Model Training

## Baseline Model

### Using Dummy Classifier

In [8]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier(strategy='most_frequent',random_state=1)
dc.fit(X_train,y_train)
dc.score(X_test,y_test)

0.07746478873239436

### As dummy isnt giving good results we will consider logistic regression as our baseline model

In [9]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=1)
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

C:\Users\Nawap Bastola\Desktop\Programming\Kaggle Comp\venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2605633802816901

## Training other models to beat the baseline model

In [10]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['prognosis'])
xg = XGBClassifier(random_state=1)
xg.fit(X_train,le.transform(y_train))
xg.score(X_test,le.transform(y_test))

0.3028169014084507

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=1)
gbc.fit(X_train,y_train)
gbc.score(X_test,y_test)

0.2605633802816901

In [13]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=1)
rfc.fit(X_train,y_train)
rfc.score(X_test,y_test)

0.2676056338028169

## As XGBoost has the highest accuracy we will finetune XGBoostClassifier

In [19]:
from sklearn.model_selection import RandomizedSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 1.0],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.5, 1.0]
}

# Create an instance of the XGBoostClassifier
model = XGBClassifier(random_state=1)

# Perform hyperparameter search using RandomizedSearchCV
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=5, scoring='accuracy')
random_search.fit(X_train, le.transform(y_train))

# Get the best hyperparameters and their corresponding score
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Hyperparameters:", best_params)
print("Best Score (Accuracy):", best_score)


Best Hyperparameters: {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 1.0}
Best Score (Accuracy): 0.30619469026548674


# Finetuned Model

In [20]:
xg_tuned = XGBClassifier(subsample=0.8, n_estimators=100, max_depth=3, learning_rate=0.01,gamma=0.1,colsample_bytree=1,random_state=1)
xg_tuned.fit(X_train,le.transform(y_train))
xg_tuned.score(X_test,le.transform(y_test))

0.30985915492957744

# Evaluation

In [22]:
from sklearn.metrics import classification_report

classification_report(le.transform(y_test),xg_tuned.predict(X_test))

'              precision    recall  f1-score   support\n\n           0       0.59      0.87      0.70        15\n           1       0.33      0.12      0.17        17\n           2       0.13      0.15      0.14        13\n           3       0.43      0.27      0.33        11\n           4       0.71      0.38      0.50        13\n           5       0.25      0.08      0.12        12\n           6       0.33      0.18      0.24        11\n           7       0.35      0.67      0.46        12\n           8       0.10      0.18      0.12        11\n           9       0.21      0.67      0.32         9\n          10       0.00      0.00      0.00        18\n\n    accuracy                           0.31       142\n   macro avg       0.31      0.33      0.28       142\nweighted avg       0.31      0.31      0.28       142\n'